In [ ]:
import torch
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import damselfly as df
import math
from scipy import integrate

home = Path.home()
results = home/'group'/'project'/'results'
snr_sweep = np.linspace(1, 10, 19)

def LoadModel(model, path):
    model.load_state_dict(torch.load(path))
    model.eval()
    
    return model
    
def GenerateSinusoid(config):

    t = torch.arange(0, config['samples'], 1) * 1 / config['sample_freq']

    signal = 1 * torch.exp(1j * 2 * math.pi * t * config['freq'])

    return signal

def CalculateNoiseVar(signal, config):

    var = torch.vdot(signal, signal) / config['snr'] ** 2

    return var

def GenerateDataTensor(signal, config):

    shape = (
        config['number_signals'] + config['number_noise'],
        config['number_channel'],
        config['samples']
        )

    data = torch.zeros(shape, dtype=torch.float)

    data[0:config['number_signals'], 0, :] = signal.real
    data[0:config['number_signals'], 1, :] = signal.imag

    return data

def GenerateTargetTensor(config):

    labels = torch.zeros(
        config['number_signals'] + config['number_noise'],
        dtype=torch.long
        )

    labels[0:config['number_signals']] = 1

    return labels

def AddNoise(data, signal, config):

    shape = data.shape
    
    noise = torch.normal(
        mean = 0,
        std = math.sqrt(CalculateNoiseVar(signal, config) / 2),
        size = shape
        )

    return data + noise

def NormBatch(batch):
    
    #print(torch.max(batch[:, 0, :], -1, keepdim=True)[0])

    for ich in range(batch.shape[1]):
    
        batch[:, ich, :] *= 1 / torch.max(abs(batch[:, ich, :]), -1, keepdim=True)[0]
        #batch[:, 1, :] *= 1 / torch.max(abs(batch[:, 1, :]), -1, keepdim=True)[0]
        
    return batch  

def EvalModel(config):

    signal = GenerateSinusoid(config)

    data = GenerateDataTensor(signal, config)
    target = GenerateTargetTensor(config)

    dataset = torch.utils.data.TensorDataset(data, target)
    loader = torch.utils.data.DataLoader(
        dataset, 
        config['training']['batchsize'],
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )

    if torch.cuda.is_available():
        print('Found GPU')
        model = LoadModel(
            config['model'](
                config['model_config']['nclass'],
                config['model_config']['nch'],
                config['model_config']['conv'],
                config['model_config']['lin']
            ),
            config['saved_model']
        ).cuda(0)
    else:
        print('No GPU')
        model = LoadModel(
            config['model'](
                config['model_config']['nclass'],
                config['model_config']['nch'],
                config['model_config']['conv'],
                config['model_config']['lin']
            ),
            config['saved_model']
        ).cpu()
        
    output_list = []
    label_list = []
    with torch.no_grad():
        batch_count = 0
        for batch, labels in loader:

            batch = AddNoise(batch, signal, config)
            batch = NormBatch(batch)
            label_list.append(labels.numpy())

            if torch.cuda.is_available():
                output = model(batch.cuda(0))
                output = output.cpu()
                #output = torch.nn.functional.softmax(output.cpu(), dim=1)
                output_list.extend(output.numpy())
            else:
                output = model(batch)
                output = torch.nn.functional.softmax(output, dim=1)
                output_list.extend(output.numpy())
            
            batch_count += 1
            print(batch_count)

    return np.array(output_list), np.array(label_list).flatten()


# plot loss curves

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for snr in [1.0, 2.0, 3.0, 5.0, 10.0]:
    loss_data = np.load(results/'machine_learning'/'dnn'/'triggering'/'loss'/'toy_sinusoid'/'220223_snr_sweep'/f'model_cnn_snr{snr}.npy')
    ax.plot(loss_data[:, 2], label=f'SNR = {snr}')
    #print(loss_data)
    
ax.set_xticks(np.linspace(1, 1200, 7))
ax.set_xticklabels(np.int32(np.linspace(1, 300, 7)))
ax.set_xlabel('Training Epoch')
ax.set_ylabel('Training Loss (AU)')
ax.legend(loc=1)
ax.set_title('Model Training Loss')

plt.tight_layout()
name = '220224_model_train_loss_vs_snr.png'
save_path = Path.home()/'group'/'project'/'plots'/'machine_learning'/'dnn'/'triggering'/'toy_sinusoid'/name

plt.savefig(save_path)


# Evaluate Models

In [ ]:
conv_layer_config = [
                        [
                            [2 * 1, 16],
                            [16, 16],
                            [16, 16],
                            [1, 1,], # dilation
                            16
                        ],
                        [
                            [16, 32,],
                            [32, 32,],
                            [8, 8,],
                            [1, 1,],
                            8
                        ],
                        [
                            [32, 64,],
                            [64, 64,],
                            [4, 4,],
                            [1, 1,],
                            4
                        ],
                    ]

config = {
    'freq':50e6, # 0 to 100 MHz
    'snr': 2, # mf snr
    'samples': 8192,
    'sample_freq': 200e6,
    'number_signals': 20000,
    'number_noise': 20000,
    'number_channel': 2, # real, imag.

    'training': {
        'batchsize': 2500,
    },
    
    'model': df.models.DFCNN,
    'saved_model': results/'machine_learning'/'dnn'/'triggering'/'models'/'toy_sinusoid'/'220223_snr_sweep'/f'model_cnn_snr2.0.pt',
    'model_config': {
        'conv': conv_layer_config,
        'nclass': 2,
        'nch': 2,
        'lin': [
            [df.models.GetConv1DOutputSize(conv_layer_config, 2, 8192), 512, 256], # input dense layer sizes
            [512, 256, 128], # output dense layer sizes
            [0.0, 0.0, 0.0] # dropout
            ],
    },
}

output_list = []
target_list = []
for snr in snr_sweep:
    print(snr)
    config['snr'] = snr
    config['saved_model'] = results/'machine_learning'/'dnn'/'triggering'/'models'/'toy_sinusoid'/'220223_snr_sweep'/f'model_cnn_snr{snr}.pt'
    
    output, target = EvalModel(config)
    
    output_list.append(output)
    target_list.append(target)
    
output = np.array(output_list)
target = np.array(target_list)
    

# plot histograms of model outputs

In [ ]:
n_snr = 2

signal_inds = np.argwhere(target[0, :] == 1).squeeze()
noise_inds = np.argwhere(target[0, :] == 0).squeeze()

sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

hist = ax.hist(output[n_snr, signal_inds, 1], bins=100, density=True )
hist = ax.hist(output[n_snr, noise_inds, 1], bins=100, density=True )

# calculate pdf and cdf vs snr

In [ ]:

bins = np.linspace(-30, 30, 25001)

pdf_sig = np.zeros((snr_sweep.size, bins.size-1))
pdf_noise = np.zeros((snr_sweep.size, bins.size-1))

cdf_sig = np.zeros((snr_sweep.size, bins.size-1))
cdf_noise = np.zeros((snr_sweep.size, bins.size-1))

for i in range(len(snr_sweep)):
    print(i)
    #if i > 2:
    #    continue
        
    signal_inds = np.argwhere(target[i, :] == 1).squeeze()
    noise_inds = np.argwhere(target[i, :] == 0).squeeze()
    
    hist_sig = np.histogram(output[i, signal_inds, 1], bins=bins, density=True )
    hist_noise = np.histogram(output[i, noise_inds, 1], bins=bins, density=True )
    
    hist_sig = hist_sig[0], hist_sig[1][1:]
    hist_noise = hist_noise[0], hist_noise[1][1:]
    
    pdf_sig[i, :] = hist_sig[0]
    pdf_noise[i, :] = hist_noise[0]

    #print(hist)
    for j in range(len(bins)-1):
        #if j % 250 == 249:
        #    print(j+1)
        cdf_sig[i, j] = integrate.trapezoid(hist_sig[0][0:j+1], x=hist_sig[1][0:j+1])
        cdf_noise[i, j] = integrate.trapezoid(hist_noise[0][0:j+1], x=hist_noise[1][0:j+1])    


# save pdf and cdf vs snr

In [ ]:
name = '220224_cnn_pdf_and_cdf_vs_snr.npz'
save = results/'machine_learning'/'dnn'/'triggering'/'eval_outputs'/'toy_sinusoid'/'220223_snr_sweep'/name

np.savez(
    save,
    pdf_signal=pdf_sig,
    pdf_noise=pdf_noise,
    cdf_signal=cdf_sig, 
    cdf_noise=cdf_noise,
    bins=bins[1:],
    snr=snr_sweep
)

In [ ]:
plt.plot(bins[1:], cdf_noise[5, :])
plt.plot(bins[1:], cdf_sig[5, :])

In [ ]:
plt.plot(bins[1:], pdf_noise[5, :])
plt.plot(bins[1:], pdf_sig[5, :])

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for i in np.arange(0, 18, 2):
    ax.plot(1-cdf_noise[i, :], 1-cdf_sig[i, :])
    
ax.plot(np.linspace(0, 1, 1001), np.linspace(0, 1, 1001), color='grey', linestyle='--')

ax.set_xscale('log')
ax.set_xlim(1e-6, 1)



In [ ]:
print(np.argmin(abs(1-cdf_noise[0] - 1e-4)))

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

select_fpr = 1e-3
select_tpr_vs_snr = np.zeros(snr_sweep.size)
for i in range(snr_sweep.size):
    index_near_fpr = np.argmin(abs(1-cdf_noise[i, :] - select_fpr))
    
    select_tpr_vs_snr[i] = 1-cdf_sig[i, index_near_fpr]
    
    
ax.plot(snr_sweep, select_tpr_vs_snr, '-')

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for fpr in [1e-1, 1e-2, 1e-3, 1e-4]:
    select_fpr = fpr
    select_tpr_vs_snr = np.zeros(snr_sweep.size)
    for i in range(snr_sweep.size):
        index_near_fpr = np.argmin(abs(1-cdf_noise[i, :] - select_fpr))

        select_tpr_vs_snr[i] = 1-cdf_sig[i, index_near_fpr]


    ax.plot(snr_sweep, select_tpr_vs_snr, '-')